In [2]:
from pathlib import Path
import os
import json
def read_data(fname: str):
    """
    Reads a JSON file containing output.
    Each maze is a list of ROWS lists, each with ROWS int values (0=free, 1=blocked).
    Returns a list of maze[r][c] grids.
    """
    ROOT = Path.cwd()
    #print(ROOT)
    DATA = ROOT / fname

    with open(fname, "r", encoding="utf-8") as fp:
        data = json.load(fp)
    return data


test = read_data("results_q2.json")
print(test)

[{'maze_id': 0, 'max_g': {'found': True, 'path_length': 354, 'expanded': 12016, 'replans': 132, 'runtime_ms': 53.15910000012991}, 'min_g': {'found': True, 'path_length': 362, 'expanded': 363996, 'replans': 136, 'runtime_ms': 1423.1290000000172}}, {'maze_id': 1, 'max_g': {'found': False, 'path_length': -1, 'expanded': 24112, 'replans': 162, 'runtime_ms': 100.84290000008878}, 'min_g': {'found': False, 'path_length': -1, 'expanded': 421667, 'replans': 183, 'runtime_ms': 1625.668899999937}}, {'maze_id': 2, 'max_g': {'found': False, 'path_length': -1, 'expanded': 25419, 'replans': 193, 'runtime_ms': 107.88249999995969}, 'min_g': {'found': False, 'path_length': -1, 'expanded': 326883, 'replans': 188, 'runtime_ms': 1251.4022000000296}}, {'maze_id': 3, 'max_g': {'found': False, 'path_length': -1, 'expanded': 30017, 'replans': 229, 'runtime_ms': 124.91180000006352}, 'min_g': {'found': False, 'path_length': -1, 'expanded': 520325, 'replans': 231, 'runtime_ms': 1850.090899999941}}, {'maze_id': 4,

In [12]:
def analysis_q2(result_file):
    """
    Display a table for analysis of q2
    """
    # Ensure mazes exist or generate them
    # Compare the results of breaking Repeated foward A* ties by smallest g values vs largest g values
    results = []
    num_mazes = 50
    print(f"Displaying max_g VS min_g output for 50 mazes...")
    print(f"{'Maze':<10} | {'Solvable':<10} | {'Cells expanded by largest g'} | {'Cells expanded by Smallest g'} | {'Max Replan':<10} | {'Min Replan':<10} | {'Max Len':<10} | {'Min Len':<10}")
    print("-" * 135)
    # retrieve list of maze outputs
    q2_res = read_data(result_file)
    
    for maze in q2_res:
        maze_name = f"maze{maze["maze_id"]}"
        solvable = "Discrepancy"
        if maze["max_g"]["found"] == maze["min_g"]["found"]:
            solvable = str(maze["max_g"]["found"])
        max_length = maze["max_g"]["path_length"]
        min_length = maze["min_g"]["path_length"]
        if solvable == "False":
            max_length = 0
            min_length = 0

        #get cell expansion count
        forward_largest_g_exp = maze["max_g"]["expanded"]
        forward_smallest_g_exp = maze["min_g"]["expanded"]

        #get cell replans
        largest_g_replans = maze["max_g"]["replans"]
        smallest_g_replans = maze["min_g"]["replans"]


        results.append({
            "Maze": maze_name,
            "Solvable": solvable,
            "Fwd_Largest_g": forward_largest_g_exp,
            "Fwd_Smallest_g": forward_smallest_g_exp,
            "Max_g_replan": largest_g_replans,
            "Min_g_replan": smallest_g_replans,
            "Max_length": max_length,
            "Min_length": min_length
        })

        print(f"{maze_name:<10} | {solvable:<10} | {forward_largest_g_exp:<27.2f} | {forward_smallest_g_exp:<28.2f} | {largest_g_replans:<10} | {smallest_g_replans:<10} | {max_length:<10} | {min_length:<10}")

    # Calculate averages
    avg_fwd_large = sum(r["Fwd_Largest_g"] for r in results) / num_mazes
    avg_fwd_small = sum(r["Fwd_Smallest_g"] for r in results) / num_mazes
    avg_largest_time = sum(r["Max_g_replan"] for r in results) / num_mazes
    avg_smallest_time = sum(r["Min_g_replan"] for r in results) / num_mazes
    avg_max_length = sum(r["Max_length"] for r in results) / num_mazes
    avg_min_length = sum(r["Min_length"] for r in results) / num_mazes
    
    print("-" * 135)
    print(f"{'Average':<10} | {'------':<10} | {avg_fwd_large:<27.2f} | {avg_fwd_small:<28.2f} | {avg_largest_time:<10.4f} | {avg_smallest_time:<10.4f} | {avg_max_length:<10.4f} | {avg_min_length:<10.4f} ")

    #Calculate how better prioritizing by large is in terms of % improvement
    large_cell_improvement = (avg_fwd_large - avg_fwd_small)/avg_fwd_small
    large_replan_improvement = (avg_largest_time - avg_smallest_time)/avg_smallest_time
    max_len_improvement = (avg_max_length - avg_min_length)/avg_min_length
    #% shown will be negative, which means a reduction in time (good)

    print(f"{'Improved':<10} | {'------':<10} | {large_cell_improvement:<27.2f} | {'------':<28} | {large_replan_improvement:<10.4f} | {'------':<10} | {max_len_improvement:<10.4f} | {'------':<10}")

analysis_q2("results_q2.json")

Displaying max_g VS min_g output for 50 mazes...
Maze       | Solvable   | Cells expanded by largest g | Cells expanded by Smallest g | Max Replan | Min Replan | Max Len    | Min Len   
---------------------------------------------------------------------------------------------------------------------------------------
maze0      | True       | 12016.00                    | 363996.00                    | 132        | 136        | 354        | 362       
maze1      | False      | 24112.00                    | 421667.00                    | 162        | 183        | 0          | 0         
maze2      | False      | 25419.00                    | 326883.00                    | 193        | 188        | 0          | 0         
maze3      | False      | 30017.00                    | 520325.00                    | 229        | 231        | 0          | 0         
maze4      | False      | 21945.00                    | 346138.00                    | 137        | 138        | 0          | 0   

In [17]:
def analysis_q3():
    """
    Display a table for analysis of q3
    """
    results = []
    num_mazes = 50
    result_file = 'results_q3.json'
    print(f"Displaying forward VS backward output for 50 mazes...")
    print(f"{'Maze':<10} | {'Solvable':<10} | {'Cells expanded by forward':27} | {'Cells expanded by backward':28} | {'Fwd Replan':<10} | {'Bwd Replan':<10} | {'Fwd Len':<10} | {'Bwd Len':<10}")
    print("-" * 135)
    # retrieve list of maze outputs
    q2_res = read_data(result_file)
    
    for maze in q2_res:
        maze_name = f"maze{maze["maze_id"]}"
        solvable = "Discrepancy"
        if maze["fwd"]["found"] == maze["bwd"]["found"]:
            solvable = str(maze["fwd"]["found"])
        max_length = maze["fwd"]["path_length"]
        min_length = maze["bwd"]["path_length"]
        if solvable == "False":
            max_length = 0
            min_length = 0

        #get cell expansion count
        forward_largest_g_exp = maze["fwd"]["expanded"]
        backward_smallest_g_exp = maze["bwd"]["expanded"]

        #get cell replans
        fwd_g_replans = maze["fwd"]["replans"]
        bwd_g_replans = maze["bwd"]["replans"]


        results.append({
            "Maze": maze_name,
            "Solvable": solvable,
            "Fwd_Largest_g": forward_largest_g_exp,
            "Bwd_Smallest_g": backward_smallest_g_exp,
            "Fwd_g_replan": fwd_g_replans,
            "Bwd_g_replan": bwd_g_replans,
            "Fwd_length": max_length,
            "Bwd_length": min_length
        })

        print(f"{maze_name:<10} | {solvable:<10} | {forward_largest_g_exp:<27.2f} | {backward_smallest_g_exp:<28.2f} | {fwd_g_replans:<10} | {bwd_g_replans:<10} | {max_length:<10} | {min_length:<10}")

    # Calculate averages
    avg_fwd_large = sum(r["Fwd_Largest_g"] for r in results) / num_mazes
    avg_fwd_small = sum(r["Bwd_Smallest_g"] for r in results) / num_mazes
    avg_largest_time = sum(r["Fwd_g_replan"] for r in results) / num_mazes
    avg_smallest_time = sum(r["Bwd_g_replan"] for r in results) / num_mazes
    avg_max_length = sum(r["Fwd_length"] for r in results) / num_mazes
    avg_min_length = sum(r["Bwd_length"] for r in results) / num_mazes
    
    print("-" * 135)
    print(f"{'Average':<10} | {'------':<10} | {avg_fwd_large:<27.2f} | {avg_fwd_small:<28.2f} | {avg_largest_time:<10.4f} | {avg_smallest_time:<10.4f} | {avg_max_length:<10.4f} | {avg_min_length:<10.4f} ")

    #Calculate how better prioritizing by large is in terms of % improvement
    large_cell_improvement = (avg_fwd_large - avg_fwd_small)/avg_fwd_small
    large_replan_improvement = (avg_largest_time - avg_smallest_time)/avg_smallest_time
    max_len_improvement = (avg_max_length - avg_min_length)/avg_min_length
    #% shown will be negative, which means a reduction in time (good)

    print(f"{'Improved':<10} | {'------':<10} | {large_cell_improvement:<27.2f} | {'------':<28} | {large_replan_improvement:<10.4f} | {'------':<10} | {max_len_improvement:<10.4f} | {'------':<10}")

analysis_q3()

Displaying forward VS backward output for 50 mazes...
Maze       | Solvable   | Cells expanded by forward   | Cells expanded by backward   | Fwd Replan | Bwd Replan | Fwd Len    | Bwd Len   
---------------------------------------------------------------------------------------------------------------------------------------
maze0      | True       | 12016.00                    | 118685.00                    | 132        | 138        | 354        | 360       
maze1      | False      | 24112.00                    | 161641.00                    | 162        | 175        | 0          | 0         
maze2      | False      | 25419.00                    | 119624.00                    | 193        | 182        | 0          | 0         
maze3      | False      | 30017.00                    | 268935.00                    | 229        | 242        | 0          | 0         
maze4      | False      | 21945.00                    | 150370.00                    | 137        | 123        | 0          |

In [19]:
def analysis_q5():
    """
    Display a table for analysis of q5
    """
    results = []
    num_mazes = 50
    result_file = 'results_q5.json'
    print(f"Displaying adaptive VS forward output for 50 mazes...")
    print(f"{'Maze':<10} | {'Solvable':<10} | {'Cells expanded by forward':27} | {'Cells expanded by adaptive':28} | {'Fwd Replan':<10} | {'Adp Replan':<10} | {'Fwd Len':<10} | {'Adp Len':<10}")
    print("-" * 135)
    # retrieve list of maze outputs
    q2_res = read_data(result_file)
    
    for maze in q2_res:
        maze_name = f"maze{maze["maze_id"]}"
        solvable = "Discrepancy"
        if maze["fwd"]["found"] == maze["adaptive"]["found"]:
            solvable = str(maze["fwd"]["found"])
        max_length = maze["fwd"]["path_length"]
        min_length = maze["adaptive"]["path_length"]
        if solvable == "False":
            max_length = 0
            min_length = 0

        #get cell expansion count
        forward_largest_g_exp = maze["fwd"]["expanded"]
        adaptive_smallest_g_exp = maze["adaptive"]["expanded"]

        #get cell replans
        fwd_g_replans = maze["fwd"]["replans"]
        adaptive_g_replans = maze["adaptive"]["replans"]


        results.append({
            "Maze": maze_name,
            "Solvable": solvable,
            "Fwd_Largest_g": forward_largest_g_exp,
            "adaptive_g": adaptive_smallest_g_exp,
            "Fwd_g_replan": fwd_g_replans,
            "adaptive_replan": adaptive_g_replans,
            "Fwd_length": max_length,
            "adaptive_length": min_length
        })

        print(f"{maze_name:<10} | {solvable:<10} | {forward_largest_g_exp:<27.2f} | {adaptive_smallest_g_exp:<28.2f} | {fwd_g_replans:<10} | {adaptive_g_replans:<10} | {max_length:<10} | {min_length:<10}")

    # Calculate averages
    avg_fwd_large = sum(r["Fwd_Largest_g"] for r in results) / num_mazes
    avg_fwd_small = sum(r["adaptive_g"] for r in results) / num_mazes
    avg_largest_time = sum(r["Fwd_g_replan"] for r in results) / num_mazes
    avg_smallest_time = sum(r["adaptive_replan"] for r in results) / num_mazes
    avg_max_length = sum(r["Fwd_length"] for r in results) / num_mazes
    avg_min_length = sum(r["adaptive_length"] for r in results) / num_mazes
    
    print("-" * 135)
    print(f"{'Average':<10} | {'------':<10} | {avg_fwd_large:<27.2f} | {avg_fwd_small:<28.2f} | {avg_largest_time:<10.4f} | {avg_smallest_time:<10.4f} | {avg_max_length:<10.4f} | {avg_min_length:<10.4f} ")

    #Calculate how better prioritizing by large is in terms of % improvement
    large_cell_improvement = (avg_fwd_large - avg_fwd_small)/avg_fwd_small
    large_replan_improvement = (avg_largest_time - avg_smallest_time)/avg_smallest_time
    max_len_improvement = (avg_max_length - avg_min_length)/avg_min_length
    #% shown will be negative, which means a reduction in time (good)

    print(f"{'Improved':<10} | {'------':<10} | {large_cell_improvement:<27.2f} | {'------':<28} | {large_replan_improvement:<10.4f} | {'------':<10} | {max_len_improvement:<10.4f} | {'------':<10}")

analysis_q5()

Displaying adaptive VS forward output for 50 mazes...
Maze       | Solvable   | Cells expanded by forward   | Cells expanded by adaptive   | Fwd Replan | Adp Replan | Fwd Len    | Adp Len   
---------------------------------------------------------------------------------------------------------------------------------------
maze0      | True       | 12016.00                    | 12294.00                     | 132        | 136        | 354        | 362       
maze1      | False      | 24112.00                    | 25249.00                     | 162        | 181        | 0          | 0         
maze2      | False      | 25419.00                    | 24532.00                     | 193        | 188        | 0          | 0         
maze3      | False      | 30017.00                    | 31627.00                     | 229        | 252        | 0          | 0         
maze4      | False      | 21945.00                    | 22401.00                     | 137        | 141        | 0          |